In [3]:
import numpy as np
import os
import sys
from fractions import gcd
from numbers import Number

sys.path.append('..')

import torch
from torch import Tensor, nn
from torch.nn import functional as F

from data import ArgoDataset, collate_fn
from utils import gpu, to_long,  Optimizer, StepLR, Logger, load_pretrain

from layers import Conv1d, Res1d, Linear, LinearRes, Null
from numpy import float64, ndarray
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import argparse
import random
import sys
import time
import shutil
from importlib import import_module
from numbers import Number
from torch.utils.data import Sampler, DataLoader

from lanegcn import get_model, ActorNet, MapNet, A2M, M2M, M2A, A2A,actor_gather,graph_gather

In [4]:
config, Dataset, collate_fn, net, loss, post_process, opt = get_model()

In [5]:
def worker_init_fn(pid):
    np_seed = int(pid)
    np.random.seed(np_seed)
    random_seed = np.random.randint(2 ** 32 - 1)
    random.seed(random_seed)

dataset = Dataset('./dataset/train_mini/data', config, train=True)
train_loader = DataLoader(
        dataset,
        batch_size=config["batch_size"],
        num_workers=config["workers"],
        shuffle=False,   # True: At each epoch, reorder the data
        collate_fn=collate_fn,
        pin_memory=True,
        worker_init_fn=worker_init_fn,   # The next 36 were thrown away
        drop_last=True,
    )

In [6]:
data = {}
for i, data in enumerate(train_loader):
    data = dict(data)
    break

print(data.keys())

dict_keys(['city', 'orig', 'gt_preds', 'has_preds', 'theta', 'rot', 'feats', 'ctrs', 'graph', 'trajs2', 'traj1'])


In [9]:
actor_net = ActorNet(config).cuda()
map_net = MapNet(config)
a2m = A2M(config)
m2m = M2M(config)
m2a = M2A(config)
a2a = A2A(config)

In [11]:
actors, actor_idcs = actor_gather(data["feats"])
actor_ctrs = data["ctrs"]
actors = actor_net(actors.cuda())
# construct map features
graph = graph_gather(to_long(data["graph"]))
nodes, node_idcs, node_ctrs = map_net(graph)


In [ ]:
feat = self.att[i](
                feat,
                graph["idcs"],
                graph["ctrs"],
                actors,
                actor_idcs,
                actor_ctrs,
                self.config["actor2map_dist"],
            )

In [14]:
norm = "GN"
ng = 1
n_ctx = 128
n_agt = 128

dist = nn.Sequential(
    nn.Linear(2, n_ctx),
    nn.ReLU(inplace=True),
    Linear(n_ctx, n_ctx, norm=norm, ng=ng),
)

query = Linear(n_agt, n_ctx, norm=norm, ng=ng)

ctx = nn.Sequential(
    Linear(3 * n_ctx, n_agt, norm=norm, ng=ng),
    nn.Linear(n_agt, n_agt, bias=False),
)

agt = nn.Linear(n_agt, n_agt, bias=False)
norm = nn.GroupNorm(gcd(ng, n_agt), n_agt)
linear = Linear(n_agt, n_agt, norm='GN', ng=ng, act=False)
relu = nn.ReLU(inplace=True)

/home/zzyu/anaconda3/envs/lanegcn/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: fractions.gcd() is deprecated. Use math.gcd() instead.
